In [1]:
from __future__ import print_function
from __future__ import division
import tensorflow as tf
import scipy.io as sio
import numpy as np
from datetime import datetime
from pprint import pprint

In [2]:
from sklearn.cross_validation import train_test_split

/Users/Steffi/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# Import the data for the model
A =  sio.loadmat('./HW2/A2P2.mat')['A']
X = A[:,0:-1]
Y = A[:,-1].reshape(10000,1)
# Split the data into   90% and test 10%
trainX,testx,trainY,testy = train_test_split(X,Y,test_size = 0.1)

# Set model parameters
num_features = trainX.shape[1] # num of input nodes
num_labels = trainY.shape[1] # num of outcomes
num_samples = trainX.shape[0] # 10000
h_size = 5 # num hidden layers
# Training parameters
training_epochs = 10000
n_hidden_1 = 100 # number of neurons layer_1
n_hidden_2 = 80 # number of neurons layer_2
n_hidden_3 = 50 # number of neurons layer_3
n_hidden_4 = 10 # number of neurons layer_4
n_hidden_5 = 1 # number of neurons layer_5

# Interval for each print statement
display_step = 500
global_step = tf.Variable(0, trainable=False)
# Learning rate of the network
# learning_rate = tf.train.exponential_decay(learning_rate=0.01,
#                                           global_step= global_step,
#                                           decay_steps=num_samples,
#                                           decay_rate= 0.95,
#                                           staircase=False)
learning_rate = 0.01

# Set the directory to specify where the summary will be written
summaries_dir = "./HW2/SummaryDir" + datetime.now().strftime('%Y%m%d_%H_%M_%S')

# pprint(trainX[:,1])

In [4]:
# Placeholder values to hold values from trainX and trainY
x_ = tf.placeholder(tf.float32,[None,num_features])
y_ = tf.placeholder(tf.float32,[None,num_labels])

In [5]:
# This function defines a five layered perceptron and with a sigmoid
# activation for each hidden layer
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.sigmoid(layer_1)
    
    # Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.sigmoid(layer_2)
    
    # Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3 = tf.sigmoid(layer_3)
    
    # Hidden layer with sigmoid activation
    layer_4 = tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4 = tf.sigmoid(layer_4)
     
    # Hidden layer with sigmoid activation
    layer_5 = tf.add(tf.matmul(layer_4,weights['h5']),biases['b5'])
    layer_5 = tf.sigmoid(layer_5)
#     Return layer_5 as the output layer of the neural network
    return layer_5

In [6]:
# Store layers' weight & biases, currently initialized as random normal 
# nummbers
weights = {
    'h1': tf.Variable(tf.random_normal([num_features, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(tf.random_normal([n_hidden_3, n_hidden_4])),
    'h5': tf.Variable(tf.random_normal([n_hidden_4, n_hidden_5])),
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'b5': tf.Variable(tf.random_normal([n_hidden_5])),
}

In [7]:
# Construct model
pred = multilayer_perceptron(x_, weights, biases)
# Define loss and optimizer
# Use a sigmoid loss function and GradientDescentOptimizer for the
# optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, targets=y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
# Initializing the variables
init = tf.global_variables_initializer()
# Check training correctness
correct_prediction = tf.equal(tf.round(pred),y_)
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


In [8]:
# Train the network
import timeit
# def runFnc():
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        batch_sz=100
        accr = []
    #   data size
        total_len=np.shape(trainX)[0]
    # total_batch for cost calculation
        total_batch = int(total_len/batch_sz)
        for i in xrange(0,total_len,batch_sz):
    # record the accuracy and average cost at each epoch
            acc,_,c = sess.run([accuracy,optimizer,cost],\
                               feed_dict={x_:trainX[i:i+batch_sz],\
                                          y_:trainY[i:i+batch_sz]})
            avg_cost += c/total_batch
            accr.append(acc)
        acc = np.mean(accr)
    #  print out the epoch, cost and accuracy after every 500 iterations
        if epoch % display_step == 0:
            print('Epoch: {0}, Cost: {1:.9f}, Accuracy: {2:.4f}'.format(epoch,avg_cost,acc))
    print("Done with training!!!")
    #Evaluate performance with the test data
    print("final cost on test set: %s" %str(sess.run(accuracy, feed_dict={x_:testx, y_:testy})))
# timeit.timeit(runFnc())

Epoch: 0, Cost: 0.769487671, Accuracy: 0.4878
Epoch: 500, Cost: 0.693425459, Accuracy: 0.5587
Epoch: 1000, Cost: 0.693284265, Accuracy: 0.5587
Epoch: 1500, Cost: 0.692998490, Accuracy: 0.5587
Epoch: 2000, Cost: 0.634441923, Accuracy: 0.7768
Epoch: 2500, Cost: 0.588036466, Accuracy: 0.8610
Epoch: 3000, Cost: 0.579018646, Accuracy: 0.8776
Epoch: 3500, Cost: 0.575209681, Accuracy: 0.8829
Epoch: 4000, Cost: 0.573128091, Accuracy: 0.8853
Epoch: 4500, Cost: 0.571741788, Accuracy: 0.8873
Epoch: 5000, Cost: 0.570734091, Accuracy: 0.8887
Epoch: 5500, Cost: 0.569979316, Accuracy: 0.8901
Epoch: 6000, Cost: 0.569396356, Accuracy: 0.8908
Epoch: 6500, Cost: 0.568931543, Accuracy: 0.8916
Epoch: 7000, Cost: 0.568548781, Accuracy: 0.8919
Epoch: 7500, Cost: 0.568225133, Accuracy: 0.8923
Epoch: 8000, Cost: 0.567946589, Accuracy: 0.8928
Epoch: 8500, Cost: 0.567706405, Accuracy: 0.8931
Epoch: 9000, Cost: 0.567498373, Accuracy: 0.8933
Epoch: 9500, Cost: 0.567317763, Accuracy: 0.8937
Done with training!!!
fi

In [ ]:
bb = trainY[i:i+batch_sz]

In [ ]:
np.shape(bb)

In [ ]:
# Close the session and exit
sess.close()